In [ ]:
!pip install simpletransformers
!pip install jsonlines

In [ ]:
import jsonlines
import pandas as pd

d =[]

with jsonlines.open('train.jsonl') as f:


    for line in f.iter():
        d.append(line)

columns  = d[0].keys()

train = pd.DataFrame(columns=columns)
for i in d:
    train.loc[len(train.index)] = list(i.values())

In [ ]:
d =[]

with jsonlines.open('validation.jsonl') as f:

    for line in f.iter():
        d.append(line)
columns  = d[0].keys()

valid = pd.DataFrame(columns=columns)
for i in d:
    valid.loc[len(valid.index)] = list(i.values())

In [ ]:
valid.head(1)

In [ ]:
train.head(1)

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [ ]:
model_args = ClassificationArgs(num_train_epochs=6)

# Create a ClassificationModel
model = ClassificationModel(
    'deberta',
    'debert-base-cased',
    num_labels=3,
    args=model_args,


)

In [ ]:
train.rename(columns= {'tags':'spoilerType'},inplace=True)

In [ ]:
train.head(1)

In [ ]:
ret=[]
c =0
for _, i in list(train.iterrows()):
        try:
            # print(''.join(i['postText']) + '-' + i['targetTitle'] + ' ' + ''.join(i['targetParagraphs']))
            ret += [{'uuid': i['uuid'] , 'text': ''.join(i['postText']) + '-' + i['targetTitle'] + ' ' + ''.join(i['targetParagraphs']) , 'labels':''.join(i['spoilerType']) }]
            ""
        except:
            ret +=  [{'text':'','uuid':i['uuid']}]
train = pd.DataFrame(ret)

In [ ]:
train.head()

In [ ]:
train= train.drop('uuid',axis=1)

In [ ]:
train.head()

In [ ]:
# uuid = tt['uuid']

In [ ]:
# def trans(data):
#     uuid = data['uuid']
#     train = data[['text','labels']]
#     model.train_model(train)



In [ ]:
def rep(a):
  if a =='passage':

    return 0
  elif a == 'phrase':
    return 1
  else:
    return 2

In [ ]:
train['labels'] = train['labels'].apply(rep)

In [ ]:
train.head()

In [ ]:
model.train_model(train)


In [ ]:
 model.predict([''' Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea-Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had A Better Idea It’ll be just like old times this weekend for Tom Brady and Wes Welker.', 'Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up.', 'But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension.', '"I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we can throw a little bit.’ "', 'Welker was one of Brady’s favorite targets for six seasons from 2007 to 2012. It’s understandable him and Brady want to meet with both being in the same area. But Brady typically is all business during football season. Welker probably should have known what he was getting into when reaching out to his buddy.', '"That’s the only thing we really have planned," Welker said of his upcoming workout with Brady. "It’s just funny. I’m sitting there trying to have dinner. ‘Hey, get your ass up here and let’s go throw.’ I’m like, ‘Aw jeez, man.’ He’s going to have me running like 2-minute drills in his backyard or something."', 'Maybe Brady will put a good word in for Welker down in Foxboro if the former Patriots wide receiver impresses him enough. '''])

In [ ]:


result, model_outputs, wrong_predictions = model.eval_model(train)


In [ ]:
result

In [ ]:
model_outputs

In [ ]:
import numpy as np
predictions = np.argmax(model_outputs,axis=1)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(list(train['labels']), predictions)) # Value between 0 and 1

print("Accuracy Percentage {} %:".format(100*accuracy_score(list(train['labels']), predictions)))

In [ ]:
def rever(a):
  if a==0:
    return 'passage'
  elif a==1:
    return 'phrase'
  else:
    return 'multi'
def main(train):
  train.rename(columns= {'tags':'spoilerType'},inplace=True)
  ret=[]
  c =0
  for _, i in list(train.iterrows()):
          try:
              # print(''.join(i['postText']) + '-' + i['targetTitle'] + ' ' + ''.join(i['targetParagraphs']))
              ret += [{'uuid': i['uuid'] , 'text': ''.join(i['postText']) + '-' + i['targetTitle'] + ' ' + ''.join(i['targetParagraphs']) , 'labels':''.join(i['spoilerType']) }]
              ""
          except:
              ret +=  [{'text':'','uuid':i['uuid']}]
  tt = pd.DataFrame(ret)
  uuid = tt['uuid']
  tt= tt.drop('uuid',axis=1)
  predictions, raw_outputs = model.predict(list(tt['text']))
  z = tt['labels'].apply(rep)

  accuracy = accuracy_score(list(z), predictions)
  uuid = pd.Series(uuid)
  z = pd.Series(predictions, name='Prediciton')
  output = pd.concat([uuid,z],axis=1)
  output['Prediciton'] = output['Prediciton'].apply(rever)



  return accuracy , output






In [ ]:
accuracy , output = main(valid)

In [ ]:
accuracy

In [ ]:
output